In [ ]:
import pandas as pd
import numpy as np
import os
import random
from scipy.stats import pearsonr,spearmanr
from scipy import stats

In [ ]:
# fdr multiple testing correction
from statsmodels.stats import multitest

def fdr_correction(P):
    size = P.shape
    temp_p = P.flatten()
    Ps = multitest.multipletests(temp_p,alpha=0.05,method='fdr_bh')
    P_corrected = Ps[1].reshape(size)

    return P_corrected

In [ ]:
# the function of calculating regional replicability
def estimate_reliability(data1,data2,t=None):
    s = data1.shape
    odata1 = np.zeros((s[0],s[1],s[2]))
    if t == 'fdr':
        for i in range(s[0]):
            for j in range(s[1]):
                corr_map1 = data1[i,j,:,:]
                corr_map2 = data2[i,j,:,:]
                correct_P1 = fdr_correction(corr_map1[:,1])
                correct_P2 = fdr_correction(corr_map2[:,1])

                tmp_data = np.zeros((s[2],2))
                for k in range(s[2]):
                    t1 = corr_map1[k,0]
                    t2 = corr_map2[k,0]
                    if t1*t2 > 0 and correct_P1[k] < 0.05 and correct_P2[k] < 0.05:
                        odata1[i,j,k] = 1
    elif t == 'bonferroni':
        for i in range(s[0]):
            for j in range(s[1]):
                corr_map1 = data1[i,j,:,:]
                corr_map2 = data2[i,j,:,:]
                
                tmp_data = np.zeros((s[2],2))
                tp = 0.05/s[2]
                for k in range(s[2]):
                    t1 = corr_map1[k,0]
                    t2 = corr_map2[k,0]
                    if t1*t2 > 0 and corr_map1[k,1] < tp and corr_map2[k,1] < tp:
                        odata1[i,j,k] = 1
    else:
        for i in range(s[0]):
            for j in range(s[1]):
                corr_map1 = data1[i,j,:,:]
                corr_map2 = data2[i,j,:,:]
                
                tmp_data = np.zeros((s[2],2))
                for k in range(s[2]):
                    t1 = corr_map1[k,0]
                    t2 = corr_map2[k,0]
                    if t1*t2 > 0 and corr_map1[k,1] < float(t) and corr_map2[k,1] < float(t):
                        odata1[i,j,k] = 1
    
    odata2 = np.zeros((s[0],s[2]))
    for i in range(s[0]):
        for j in range(s[2]):
            tmp_data2 = odata1[i,:,j]
            odata2[i,j] = np.sum(tmp_data2)/s[1]
            
    return odata2

In [ ]:
file_path = '/data/sliu/sampling_ukbb_analysis/new_results/'
files = os.listdir(file_path)

# t indictaes the significance thresholds: p<0.05, p<0.01, fdr_p<0.05, fdr_bonferroni<0.05
t = '0.05'
for f in files:
    print(f)
    CT_file_path1 = os.path.join(file_path,f) + '/random/random_data_CT1.npy'
    CT_file_path2 = os.path.join(file_path,f) + '/random/random_data_CT2.npy'
    CSA_file_path1 = os.path.join(file_path,f) + '/random/random_data_CSA1.npy'
    CSA_file_path2 = os.path.join(file_path,f) + '/random/random_data_CSA2.npy'
    FC_file_path1 = os.path.join(file_path,f) + '/random/random_data_FC1.npy'
    FC_file_path2 = os.path.join(file_path,f) + '/random/random_data_FC2.npy'
    
    random_data_CSA1 = np.load(CSA_file_path1)
    random_data_CT1 = np.load(CT_file_path1)
    random_data_FC1= np.load(FC_file_path1)
        
    random_data_CSA2 = np.load(CSA_file_path2)
    random_data_CT2 = np.load(CT_file_path2)
    random_data_FC2 = np.load(FC_file_path2)
    
    CT_reliability = estimate_reliability(random_data_CT1,random_data_CT2,t=t)
    CSA_reliability = estimate_reliability(random_data_CSA1,random_data_CSA2,t=t)
    FC_reliability = estimate_reliability(random_data_FC1,random_data_FC2,t=t)
    
    CSA_file_name = 'new_results/'+f+'/CSA_reliability_'+t+'.csv'
    CT_file_name = 'new_results/'+f+'/CT_reliability_'+t+'.csv'
    FC_file_name = 'new_results/'+f+'/FC_reliability_'+t+'.csv'

    data1 = pd.DataFrame(data=CT_reliability)
    data2 = pd.DataFrame(data=CSA_reliability)
    data3 = pd.DataFrame(data=FC_reliability)

    data1.to_csv(CT_file_name,index=False)
    data2.to_csv(CSA_file_name,index=False)
    data3.to_csv(FC_file_name,index=False)